In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template(
    "Make a quiz about {city}. each answers have five answer. one correct answer and four false answer"
)

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What is the capital city of Italy?","answers":[{"answer":"Rome","correct":true},{"answer":"Milan","correct":false},{"answer":"Florence","correct":false},{"answer":"Venice","correct":false},{"answer":"Naples","correct":false}]},{"question":"Which ancient civilization built the Colosseum in Rome?","answers":[{"answer":"Greeks","correct":false},{"answer":"Romans","correct":true},{"answer":"Egyptians","correct":false},{"answer":"Persians","correct":false},{"answer":"Mayans","correct":false}]},{"question":"Who was the first Roman Emperor?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false},{"answer":"Caligula","correct":false},{"answer":"Trajan","correct":false}]},{"question":"What river runs through the city of Rome?","answers":[{"answer":"Nile","correct":false},{"answer":"Tiber","correct":true},{"answer":"Danube","correct":false},{"answer":"Seine","correct":false},{"answer":"Thames","corr

In [4]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What is the capital city of Italy?', 'answers': [{'answer': 'Rome', 'correct': True}, {'answer': 'Milan', 'correct': False}, {'answer': 'Florence', 'correct': False}, {'answer': 'Venice', 'correct': False}, {'answer': 'Naples', 'correct': False}]}
{'question': 'Which ancient civilization built the Colosseum in Rome?', 'answers': [{'answer': 'Greeks', 'correct': False}, {'answer': 'Romans', 'correct': True}, {'answer': 'Egyptians', 'correct': False}, {'answer': 'Persians', 'correct': False}, {'answer': 'Mayans', 'correct': False}]}
{'question': 'Who was the first Roman Emperor?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}, {'answer': 'Caligula', 'correct': False}, {'answer': 'Trajan', 'correct': False}]}
{'question': 'What river runs through the city of Rome?', 'answers': [{'answer': 'Nile', 'correct': False}, {'answer': 'Tiber', 'correct': True}, {'answer': 'Danube', 'correct': F